<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introducción" data-toc-modified-id="Introducción-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introducción</a></span></li><li><span><a href="#Representaciones" data-toc-modified-id="Representaciones-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Representaciones</a></span></li><li><span><a href="#Algoritmos" data-toc-modified-id="Algoritmos-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Algoritmos</a></span></li><li><span><a href="#Métricas-de-Evaluación" data-toc-modified-id="Métricas-de-Evaluación-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Métricas de Evaluación</a></span></li><li><span><a href="#Experimentos" data-toc-modified-id="Experimentos-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Experimentos</a></span><ul class="toc-item"><li><span><a href="#Importar-librerías-y-utiles" data-toc-modified-id="Importar-librerías-y-utiles-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Importar librerías y utiles</a></span></li><li><span><a href="#Definir-métodos-de-evaluación" data-toc-modified-id="Definir-métodos-de-evaluación-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Definir métodos de evaluación</a></span></li><li><span><a href="#Datos" data-toc-modified-id="Datos-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Datos</a></span></li><li><span><a href="#Analizar-los-datos" data-toc-modified-id="Analizar-los-datos-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Analizar los datos</a></span></li><li><span><a href="#Custom-Features" data-toc-modified-id="Custom-Features-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Custom Features</a></span></li><li><span><a href="#Definir-la-representación-y-el-clasificador" data-toc-modified-id="Definir-la-representación-y-el-clasificador-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Definir la representación y el clasificador</a></span></li><li><span><a href="#Ejecutar-el-pipeline-para-algún-dataset" data-toc-modified-id="Ejecutar-el-pipeline-para-algún-dataset-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>Ejecutar el pipeline para algún dataset</a></span></li><li><span><a href="#Ejecutar-el-pipeline-por-cada-train-set" data-toc-modified-id="Ejecutar-el-pipeline-por-cada-train-set-5.8"><span class="toc-item-num">5.8&nbsp;&nbsp;</span>Ejecutar el pipeline por cada train set</a></span></li><li><span><a href="#Predecir-target-set" data-toc-modified-id="Predecir-target-set-5.9"><span class="toc-item-num">5.9&nbsp;&nbsp;</span>Predecir target set</a></span></li></ul></li><li><span><a href="#Conclusiones" data-toc-modified-id="Conclusiones-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusiones</a></span></li></ul></div>

# Tarea 1 NLP

-------------------------------

- **Nombre:**

- **Usuario o nombre de equipo en Codalab:** 




--------------------------

Este es el baseline de la tarea 1. Contiene la estructura del reporte mas un código básico que:

- Obtiene los dataset.
- Divide los datasets en train (entrenamiento y prueba) y target set (el que clasificar para subir a la competencia).
- Crea un Pipeline que: 
    - Crea features personalizadas.
    - Transforma los dataset a bag of words(BoW).  
    - Entrena un clasificador usando cada train set.
- Clasifica y evalua el sistema creado usando el test set.
- Clasifica el target set.
- Genera una submission con el target en formato zip en el directorio en donde se está ejecutando el notebook. 

La idea es que a partir de este notebook puedan empezar la tarea. Obviamente, no es requisito usarlo, pero puede facilitarles mucho al comienzo.

Para ver las instrucciones, descarga el enunciado de la tarea en ucursos. **¡Recuerda ingresar tu nombre, el de tu equipo y sus usuarios en codalab!**

(Pueden eliminar esta y cualquier celda con instrucciones...)

## Introducción

...

## Representaciones

...

## Algoritmos

...

## Métricas de Evaluación

- AUC: ...
- Kappa: ...
- Accuracy: ...


## Experimentos

...

### Importar librerías y utiles

In [1]:
import pandas as pd
import shutil

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, cohen_kappa_score, classification_report, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin


import os
import numpy as np

### Definir métodos de evaluación

Estas funciones están a cargo de evaluar los resultados de la tarea. No deberían cambiarlas.


In [2]:
def auc_score(test_set, predicted_set):
    high_predicted = np.array([prediction[2] for prediction in predicted_set])
    medium_predicted = np.array(
        [prediction[1] for prediction in predicted_set])
    low_predicted = np.array([prediction[0] for prediction in predicted_set])

    high_test = np.where(test_set == 'high', 1.0, 0.0)
    medium_test = np.where(test_set == 'medium', 1.0, 0.0)
    low_test = np.where(test_set == 'low', 1.0, 0.0)

    auc_high = roc_auc_score(high_test, high_predicted)
    auc_med = roc_auc_score(medium_test, medium_predicted)
    auc_low = roc_auc_score(low_test, low_predicted)

    auc_w = (low_test.sum() * auc_low + medium_test.sum() * auc_med +
             high_test.sum() * auc_high) / (
                 low_test.sum() + medium_test.sum() + high_test.sum())
    return auc_w


def evaulate(predicted_probabilities, y_test, labels, dataset_name):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador.
    # (que comunmente, es distinto a ['low', 'medium', 'high'])
    predicted_labels = [
        labels[np.argmax(item)] for item in predicted_probabilities
    ]

    # Confusion Matrix
    print('Confusion Matrix for {}:\n'.format(dataset_name))

    # Classification Report
    print(
        confusion_matrix(y_test,
                         predicted_labels,
                         labels=['low', 'medium', 'high']))

    print('\nClassification Report:\n')
    print(
        classification_report(y_test,
                              predicted_labels,
                              labels=['low', 'medium', 'high']))

    # Reorder predicted probabilities array.
    labels = labels.tolist()
    predicted_probabilities = predicted_probabilities[:, [
        labels.index('low'),
        labels.index('medium'),
        labels.index('high')
    ]]
    
    # Auc
    auc = round(auc_score(y_test, predicted_probabilities), 3)
    print("Scores:\n\nAUC: ", auc, end='\t')
    
    # Kappa
    kappa = round(cohen_kappa_score(y_test, predicted_labels), 3)
    print("Kappa:", kappa, end='\t')
    
    # Accuracy
    accuracy = round(accuracy_score(y_test, predicted_labels), 3)
    print("Accuracy:", accuracy)

    print('------------------------------------------------------\n')
    return np.array([auc, kappa, accuracy])

### Datos

Obtener los datasets desde el github del curso

In [3]:
train = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/anger-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/fear-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/joy-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/train/sadness-train.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'])
}

target = {
    'anger': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/anger-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'fear': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/fear-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'joy': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/joy-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE']),
    'sadness': pd.read_csv('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/assignment_1/data/target/sadness-target.txt', sep='\t', names=['id', 'tweet', 'class', 'sentiment_intensity'], na_values=['NONE'])
}

In [4]:
# Ejemplo de algunas filas aleatorias:
train['anger'].sample(10)

,id,tweet,class,sentiment_intensity
704,10704,"@MeghanEMurphy Oh gosh, if you get 800+ raging...",anger,medium
422,10422,Drop Snapchat names #bored #snap #swap #pics,anger,medium
56,10056,Absolutely fuming that some woman jumped into ...,anger,high
291,10291,#Anger or #wrath is an intense emotional respo...,anger,medium
474,10474,Puzzle investing opening portland feodal popul...,anger,medium
142,10142,i had an hour of football practice under the b...,anger,high
612,10612,Because you offended by the lies of Alexandria...,anger,medium
716,10716,Praying for the #Lord to keep #anger #hate #je...,anger,medium
303,10303,Yo there's a kid on my snap chat from LA &amp;...,anger,medium
47,10047,Still can't log into my fucking Snapchat #Snap...,anger,high


### Analizar los datos 

Imprimir la cantidad de tweets de cada dataset, según su intensidad de sentimiento. Noten que las clases están desbalanceadas. 

In [5]:
def get_group_dist(group_name, train):
    print(group_name, "\n",
          train[group_name].groupby('sentiment_intensity').count(),
          '\n---------------------------------------\n')


for dataset_name in train:
    get_group_dist(dataset_name, train)

anger 
                       id  tweet  class
sentiment_intensity                   
high                 163    163    163
low                  161    161    161
medium               617    617    617 
---------------------------------------

fear 
                       id  tweet  class
sentiment_intensity                   
high                 270    270    270
low                  288    288    288
medium               699    699    699 
---------------------------------------

joy 
                       id  tweet  class
sentiment_intensity                   
high                 195    195    195
low                  219    219    219
medium               488    488    488 
---------------------------------------

sadness 
                       id  tweet  class
sentiment_intensity                   
high                 197    197    197
low                  210    210    210
medium               453    453    453 
---------------------------------------



### Custom Features 

Para crear features personalizadas implementaremos nuestros propios Transformers (estandar de scikit para crear nuevas features entre otras cosas). Para esto:

1. Creamos nuestra clase Transformer extendiendo BaseEstimator y TransformerMixin. En este ejemplo, definiremos `CharsCountTransformer` que cuenta carácteres relevantes ('!', '?', '#') en los tweets.
2. Definios una función cómo `get_relevant_chars` que opera por cada tweet y retorna un arreglo.
3. Hacemos un override de la función `transform` en donde iteramos por cada tweet, llamamos a la función que hicimos antes y agregamos sus resultados a un arrelo. Finalmente lo retornamos.

Esto nos facilitará el trabajo mas adelante. Una Guia completa de las transformaciones predefinidas en scikit pueden encontrarla [aquí](https://scikit-learn.org/stable/data_transforms.html).



In [6]:
class CharsCountTransformer(BaseEstimator, TransformerMixin):
    def get_relevant_chars(self, tweet):
        num_hashtags = tweet.count('#')
        num_exclamations = tweet.count('!')
        num_interrogations = tweet.count('?')
        return [num_hashtags, num_exclamations, num_interrogations]

    def transform(self, X, y=None):
        chars = []
        for tweet in X:
            chars.append(self.get_relevant_chars(tweet))

        return np.array(chars)

    def fit(self, X, y=None):
        return self

In [7]:
# Veamos que sucede si ejecutamos el transformer
sample = train['anger'].sample(10).tweet
pd.DataFrame(zip(sample, CharsCountTransformer().transform(sample)))

,0,1
0,Just joined #pottermore and was sorted into HU...,"[2, 0, 0]"
1,I get so angry at people that don't know that ...,"[2, 0, 0]"
2,@kingcharles9th i Lowkey forgot you had twitte...,"[1, 0, 0]"
3,@canada4trumpnow @donlemon he has BAD TEMPERAM...,"[1, 0, 0]"
4,Im so angry 😂🙃,"[0, 0, 0]"
5,"Is it me, or is Ding wearing the look of a man...","[1, 0, 1]"
6,Petition for non-Maghrebi PoC to stop buying '...,"[0, 0, 0]"
7,@Idubbbz @LeafyIsHere I am offended,"[0, 0, 0]"
8,"Ananya just grabbed a bible, opened it, starte...","[0, 0, 1]"
9,@johnaugust @clmazin I was so looking forward ...,"[1, 0, 0]"


### Definir la representación y el clasificador

Para definir el flujo de los datos sobre nuestras transformaciones y el clasificador usaremos un `Pipeline`.

Un `Pipeline` es una lista de transformaciones y un estimador(clasificador) ubicado al final el cual define el proceso que seguiran nuestros datos. Nos permite ejecutar facilmente el mismo proceso sobre todos los datasets que usemos, simplificando así nuestra programación.

El pipeline más básico que podemos hacer es transformar el dataset a Bag of Words y después usar clasificar el BoW usando NaiveBayes:


        Dataset -> BagOfWords (llamado CountVectorizer) -> NaiveBayes

En código queda cómo:

```python
    Pipeline([('bow', CountVectorizer()), ('clf', MultinomialNB())])
```


Ahora, si queremos usar nuestra transformación para agregar las features que creamos, usaremos `FeatureUnion`. Esta simplemente concatenará los vectores resultantes de ejecutar los Transformer en un solo vector.


        Dataset -> FeatureUnion [CountVectorizer, CharsCountTransformer] -> NaiveBayes


En código queda como: 

```python
Pipeline([('features',FeatureUnion([('bow', CountVectorizer()),
                                    ('chars_count',CharsCountTransformer())])),
            ('clf', MultinomialNB())])

```





Algunas pistas sobre como mejorar el rendimiento del vectorizador y del clasificador:

- **Vectorizador**: investigar los modulos de `nltk`, en particular, `TweetTokenizer`, `mark_negation` para reemplazar los tokenizadores. También, el parámetro `ngram_range` (Ojo que el clf naive bayes no debería usarse con n-gramas, ya que rompe el supuesto de independencia). Además, implementar los atributos que crean útiles desde el listado del el enunciado. Investigar también el vectorizador tf-idf.

- **Clasificador**: investigar otros clasificadores mas efectivos que naive bayes. Estos deben poder retornar la probabilidad de pertenecia de las clases (ie: implementar la función `predict_proba`).

- **Features**: Recuerden que pueden implementar todas las features que se les ocurra! Pueden guiarse (pero no limitarse) por las que dice el enunciado.

- **Reducción de dimensionalidad**: También puede serles de ayuda. Referencias [aquí](https://scikit-learn.org/stable/modules/unsupervised_reduction.html).

- Por último, pueden encontrar mas referencias de cómo mejorar sus features, el vectorizador y el clasificador [aquí](https://affectivetweets.cms.waikato.ac.nz/benchmark/).

Recuerden que cada vez que ejecuten pipeline, deben hacerlo con una instancia distinta. De lo contrario, podrían solapar resultados.

### Ejecutar el pipeline para algún dataset

Ejecuta el pipeline sobre un dataset. Retorna el modelo ya entrenado mas sus labels asociadas.

In [8]:
def get_experiment_0_pipeline():
    return Pipeline([('features',
                      FeatureUnion([('bow', CountVectorizer()),
                                    ('chars_count', CharsCountTransformer())
                                    ])), ('clf', MultinomialNB())])

In [9]:
def run(dataset, dataset_name):

    # Dividimos el dataset en train y test.
    X_train, X_test, y_train, y_test = train_test_split(
        dataset.tweet,
        dataset.sentiment_intensity,
        shuffle=True,
        test_size=0.33)

    pipeline = get_experiment_0_pipeline()

    # Entrenamos el clasificador (Ejecuta el entrenamiento sobre todo el pipeline)
    pipeline.fit(X_train, y_train)

    # Predecimos las probabilidades de intensidad de cada elemento del set de prueba.
    predicted_probabilities = pipeline.predict_proba(X_test)

    # Obtenemos el orden de las clases aprendidas.
    learned_labels = pipeline.classes_

    # Evaluamos:
    scores = evaulate(predicted_probabilities, y_test, learned_labels, dataset_name)
    return pipeline, learned_labels, scores

### Ejecutar el pipeline por cada train set


In [10]:
classifiers = []
learned_labels_array = []
scores_array = []

# Por cada nombre_dataset, dataset en train ('anger', 'fear', 'joy', 'sadness')
for dataset_name, dataset in train.items():
    # ejecutamos el pipeline sobre el dataset
    classifier, learned_labels, scores = run(dataset, dataset_name)

    # guardamos el clasificador entrenado
    classifiers.append(classifier)

    # guardamos las labels aprendidas por el clasificador
    learned_labels_array.append(learned_labels)

    # guardamos los scores obtenidos
    scores_array.append(scores)

# print avg scores
print(
    "Average scores:\n\n",
    "Average AUC: {0:.3g}\t Average Kappa: {1:.3g}\t Average Accuracy: {2:.3g}"
    .format(*np.array(scores_array).mean(axis=0)))

Confusion Matrix for anger:

[[  7  47   0]
 [  5 193   8]
 [  0  43   8]]

Classification Report:

              precision    recall  f1-score   support

         low       0.58      0.13      0.21        54
      medium       0.68      0.94      0.79       206
        high       0.50      0.16      0.24        51

    accuracy                           0.67       311
   macro avg       0.59      0.41      0.41       311
weighted avg       0.64      0.67      0.60       311

Scores:

AUC:  0.615	Kappa: 0.133	Accuracy: 0.669
------------------------------------------------------

Confusion Matrix for fear:

[[ 22  69   2]
 [  8 211  14]
 [  2  70  17]]

Classification Report:

              precision    recall  f1-score   support

         low       0.69      0.24      0.35        93
      medium       0.60      0.91      0.72       233
        high       0.52      0.19      0.28        89

    accuracy                           0.60       415
   macro avg       0.60      0.44      0.4

### Predecir target set

In [11]:
def predict_target(dataset, classifier, labels):
    # Predecir las probabilidades de intensidad de cada elemento del target set.
    predicted = pd.DataFrame(classifier.predict_proba(dataset.tweet), columns=labels)
    # Agregar ids
    predicted['id'] = dataset.id.values
    # Reordenar las columnas
    predicted = predicted[['id', 'low', 'medium', 'high']]
    return predicted

In [12]:
predicted_target = {}

# Crear carpeta ./predictions
if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

# por cada target set:
for idx, key in enumerate(target):
    # Predecirlo
    predicted_target[key] = predict_target(target[key], classifiers[idx],
                                           learned_labels_array[idx])
    # Guardar predicciones en archivo separado
    predicted_target[key].to_csv('./predictions/{}-pred.txt'.format(key),
                                 sep='\t',
                                 header=False,
                                 index=False)

# Crear archivo zip
a = shutil.make_archive('predictions', 'zip', './predictions')

## Conclusiones

...